In [ ]:
from google.colab import drive #importing drive for access
drive.mount('/content/drive') #mounting drive so colab can access files

Mounted at /content/drive


In [ ]:
import pandas as pd #importing pandas

# reading the csv
data = pd.read_csv("/content/drive/Shareddrives/1:1_Aiden_Chavda/Dataset/train_regular.csv")
data.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,pIC50
0,0.464706,0.464706,-0.983618,0.145454,0.320598,-0.067105,-0.165424,-0.173523,-0.164536,-0.063671,...,-0.193550,-0.154471,-0.046078,0.0,-0.338373,0.0,-0.174836,-0.140372,-0.086433,7.494850
1,0.598283,0.598283,-0.440738,0.156438,-0.735756,-0.080063,0.676369,0.595241,0.677361,0.870610,...,-0.193550,-0.154471,-0.046078,0.0,-0.338373,0.0,-0.174836,5.620976,11.569665,6.593460
2,0.716029,0.716029,-1.191989,0.132417,-0.702170,-0.770074,0.510327,0.563305,0.511103,0.481326,...,-0.193550,-0.154471,-0.046078,0.0,-0.338373,0.0,-0.174836,-0.140372,-0.086433,7.657577
3,0.919866,0.919866,-0.126506,0.098845,-0.294494,-0.628832,-0.165739,-0.125476,-0.164667,-0.141528,...,-0.193550,-0.154471,-0.046078,0.0,-0.338373,0.0,-0.174836,-0.140372,-0.086433,8.045757
4,0.767761,0.767761,-1.108975,-2.657391,0.230813,0.777322,1.938188,1.876483,1.937533,1.804890,...,6.331864,-0.154471,-0.046078,0.0,-0.338373,0.0,-0.174836,-0.140372,-0.086433,4.522879


In [ ]:
X = data.loc[:, [cols for cols in list(data.columns) if cols != "pIC50"]].values
Y = data.loc[:, "pIC50"].values

5 fold CV for all Models

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def cross_validation(model, data = (X, Y), splits = 5):
  kf = KFold(n_splits = splits, shuffle=True, random_state=42)

  # Perform k-fold cross-validation
  rmse_scores = []
  r2_scores = []

  for train_index, valid_index in kf.split(data[0]):
    X_train, X_valid = data[0][train_index], data[0][valid_index]
    y_train, y_valid = data[1][train_index], data[1][valid_index]

    # Fit the KNN model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_valid)

    # Calculate Mean Squared Error (MSE)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    r2 = r2_score(y_valid, y_pred)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

  # get arrays
  rmse_set = np.array(rmse_scores)
  r2_set = np.array(r2_scores)

  print("{}".format(rmse_set) + "|" + "{}".format(r2_set) + "|" + "{}".format(rmse_set.mean()) + "|" + "{}".format(r2_set.mean()))
  # print("RMSE Scores: {}".format(rmse_set))
  # print("R2 Scores: {}".format(r2_set))
  # print("RMSE Mean: {}".format(rmse_set.mean()))
  # print("R2 Mean: {}".format(r2_set.mean()))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Use 5-Fold Cross Validation for hyper parameter tuning
# Try out diff values, find best value for parameter
for n in range(1, 21):
  # print(n, end = "")
  knn = KNeighborsRegressor(n_neighbors = n)
  cross_validation(knn)

[0.86034175 0.89963216 0.90530021 0.90646361 0.87183246]|[0.47986286 0.43441096 0.27257657 0.44760337 0.3607027 ]|0.8887140378132228|0.39903129139253973
[0.83141984 0.76890605 0.87021513 0.8365361  0.76884397]|[0.51424573 0.58684064 0.32786686 0.52954331 0.50282048]|0.8151842183012168|0.4922634036403517
[0.79345743 0.78012204 0.85927237 0.85424423 0.72326445]|[0.55759184 0.57469927 0.34466444 0.50941487 0.56002192]|0.8020721023097769|0.5092784664456568
[0.77924251 0.79650529 0.85240997 0.85892359 0.72187258]|[0.57330148 0.55664831 0.35509003 0.50402551 0.5617137 ]|0.8017907874097545|0.5101558083001231
[0.78164525 0.79720788 0.83211453 0.82650364 0.70245361]|[0.57066603 0.55586581 0.38543439 0.54075989 0.58497706]|0.7879849816420532|0.5275406361186536
[0.79088508 0.81737215 0.84804211 0.84030029 0.69482886]|[0.56045573 0.53311415 0.36168231 0.52529993 0.59393786]|0.7982856961157325|0.5148979946394748
[0.76940136 0.81560734 0.85507652 0.84721804 0.69650536]|[0.58401107 0.5351281  0.35104

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Use 5-Fold Cross Validation for hyper parameter tuning
# Try out diff values, find best value for parameter

#random_forest = RandomForestRegressor(n_estimators = 10, max_depth = 5)
for n in range(10, 101, 10):
  for d in range(1, 8):
    print(str(n) + "|" + str(d) + "|", end = "")
    random_forest = RandomForestRegressor(n_estimators = n, max_depth = d)
    cross_validation(random_forest)

10|1|[1.13214857 1.09462483 0.9528098  1.06596624 0.93798429]|[0.09929471 0.1626604  0.19422373 0.23609882 0.26000646]|1.0367067468299878|0.1904568237104622
10|2|[1.03088125 1.01712377 0.89606823 1.00816882 0.85063976]|[0.2532192  0.27703271 0.28733701 0.31669154 0.39140531]|0.9605763654665239|0.3051371536654225
10|3|[0.94251838 0.88841469 0.79769162 0.88973133 0.77658315]|[0.37575436 0.44842763 0.43522937 0.46780837 0.49276089]|0.8589878321013507|0.4439961253802811
10|4|[0.83267554 0.83689669 0.7682265  0.81179184 0.67438791]|[0.51277734 0.51054278 0.47618177 0.55696336 0.61747805]|0.7847956949003319|0.5347886621664527
10|5|[0.80671987 0.7699405  0.75560797 0.83540963 0.68404578]|[0.54267876 0.5857282  0.49324843 0.53080948 0.60644346]|0.770344747167512|0.551781667801601
10|6|[0.73532392 0.74224934 0.75147858 0.78936284 0.6687557 ]|[0.62004405 0.61499118 0.49877208 0.58110649 0.6238407 ]|0.7374340759348621|0.5877509003630357
10|7|[0.77642371 0.74712358 0.72580835 0.77717792 0.65516838

In [ ]:
import pickle
import os

MODEL_SAVE_FOLDER = "/content/drive/Shareddrives/1:1_Aiden_Chavda/Results"
MODEL_NAME = "best_model_KNN"

# Model with the lowest RMSE or highest r2
best_model = KNeighborsRegressor(n_neighbors=5)

# Fit the model on the full training dataset
best_model.fit(X, Y)

fh = open(os.path.join(MODEL_SAVE_FOLDER, MODEL_NAME), "wb")
pickle.dump(best_model, fh)
fh.close()

In [ ]:
import pickle
import os

MODEL_SAVE_FOLDER = "/content/drive/Shareddrives/1:1_Aiden_Chavda/Results"
MODEL_NAME = "best_model_random_forest"

# Model with the lowest RMSE or highest r2
best_model = RandomForestRegressor(n_estimators = 60, max_depth = 7)

# Fit the model on the full training dataset
best_model.fit(X, Y)

fh = open(os.path.join(MODEL_SAVE_FOLDER, MODEL_NAME), "wb")
pickle.dump(best_model, fh)
fh.close()